In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [4]:
df = fetch_hourly_rides(12)

2025-03-05 15:03:59,262 INFO: Initializing external client
2025-03-05 15:03:59,264 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 15:04:00,946 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (111.55s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-05 03:00:00+00:00,216,0
1,2025-03-05 03:00:00+00:00,210,0
2,2025-03-05 03:00:00+00:00,17,0
3,2025-03-05 03:00:00+00:00,11,0
4,2025-03-05 03:00:00+00:00,225,0
...,...,...,...
4279,2025-03-05 19:00:00+00:00,88,34
4280,2025-03-05 19:00:00+00:00,195,0
4281,2025-03-05 19:00:00+00:00,170,411
4282,2025-03-05 19:00:00+00:00,37,4


In [6]:
df_pred = fetch_predictions(12)

2025-03-05 15:06:00,189 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 15:06:00,199 INFO: Initializing external client
2025-03-05 15:06:00,200 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 15:06:00,954 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.41s) 


In [7]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,176,0.0,2025-03-05 19:00:00+00:00
1,27,0.0,2025-03-05 19:00:00+00:00
2,152,3.0,2025-03-05 19:00:00+00:00
3,43,61.0,2025-03-05 19:00:00+00:00
4,262,16.0,2025-03-05 19:00:00+00:00
...,...,...,...
499,22,0.0,2025-03-05 20:00:00+00:00
500,56,0.0,2025-03-05 20:00:00+00:00
501,123,0.0,2025-03-05 20:00:00+00:00
502,230,125.0,2025-03-05 20:00:00+00:00


In [8]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [9]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-05 19:00:00+00:00,119,0,0.0
1,2025-03-05 19:00:00+00:00,113,177,81.0
2,2025-03-05 19:00:00+00:00,128,0,0.0
3,2025-03-05 19:00:00+00:00,131,0,0.0
4,2025-03-05 19:00:00+00:00,221,0,0.0
...,...,...,...,...
247,2025-03-05 19:00:00+00:00,88,34,13.0
248,2025-03-05 19:00:00+00:00,195,0,0.0
249,2025-03-05 19:00:00+00:00,170,411,130.0
250,2025-03-05 19:00:00+00:00,37,4,1.0


In [10]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [11]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
101,2025-03-05 19:00:00+00:00,2,0,0.0,0.0
151,2025-03-05 19:00:00+00:00,3,0,0.0,0.0
167,2025-03-05 19:00:00+00:00,4,17,6.0,-11.0
28,2025-03-05 19:00:00+00:00,6,0,0.0,0.0
52,2025-03-05 19:00:00+00:00,7,5,1.0,-4.0
...,...,...,...,...,...
223,2025-03-05 19:00:00+00:00,259,0,1.0,1.0
175,2025-03-05 19:00:00+00:00,260,1,2.0,1.0
208,2025-03-05 19:00:00+00:00,261,78,21.0,-57.0
232,2025-03-05 19:00:00+00:00,262,98,16.0,-82.0


In [12]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-05 19:00:00+00:00,119,0,0.0,0.0
1,2025-03-05 19:00:00+00:00,113,177,81.0,-96.0
2,2025-03-05 19:00:00+00:00,128,0,0.0,0.0
3,2025-03-05 19:00:00+00:00,131,0,0.0,0.0
4,2025-03-05 19:00:00+00:00,221,0,0.0,0.0
...,...,...,...,...,...
247,2025-03-05 19:00:00+00:00,88,34,13.0,-21.0
248,2025-03-05 19:00:00+00:00,195,0,0.0,0.0
249,2025-03-05 19:00:00+00:00,170,411,130.0,-281.0
250,2025-03-05 19:00:00+00:00,37,4,1.0,-3.0


In [13]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [14]:
mae_by_hour["MAE"].mean()

26.384920634920636